# Train gradient boosting model using Bayesian optimization to find best hyperparameters

# 1. Imports

## 1.1 Packages

In [1]:
import sys

import pandas as pd


## 1.2 Options

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
sys.path.append('../src')

# from velib_prediction.pipelines.train_model.mlflow import (  # noqa: E402
#     create_mlflow_experiment,
# )
from velib_prediction.pipelines.train_model.nodes import (  # noqa: E402
    add_lags_sma,
    get_split_train_val_cv,
    train_model_bayesian_opti,
)


In [4]:
lags_to_try = [1,]

In [5]:
feat_date = "duedate"

## 1.3 Datasets

In [6]:
df_train = pd.read_parquet("../data/04_feature/df_feat_train.parquet")
df_train.sample(2)

,idx,stationcode,is_installed,capacity,numdocksavailable,numbikesavailable,mechanical,ebike,is_renting,is_returning,duedate,code_insee_commune,duedate_year,duedate_month,duedate_day,duedate_weekday,duedate_weekend
7,170411729623109,17041,1,36,16,16,12,4,1,1,2024-10-22 18:51:49+00:00,75056,2024,10,22,1,0
0,90201730657755,9020,1,21,19,2,1,1,1,1,2024-11-03 18:15:55+00:00,75056,2024,11,3,6,1


In [7]:
df_train.rename(columns={"numbikesavailable": "target"}, inplace=True)

# 2. Prepare datasets

In [8]:
# Add lags defined
df_train = add_lags_sma(df_train, lags_to_try, feat_id='stationcode', feat_date=feat_date, feat_target="target", n_shift=5)

In [9]:
df_train.sort_values(by="duedate", inplace=True)
df_train.drop(columns="duedate", inplace=True)

In [10]:
list_df = get_split_train_val_cv(df_train, n_splits=2)

In [11]:
df_train, df_valid = list_df[0][0], list_df[0][1]

In [12]:
feat_cat = [
    "is_installed",
    "is_renting",
    "is_returning",
    "code_insee_commune",
    "duedate_weekend",
]

# 3. Train model

In [13]:
# experiment_id = create_mlflow_experiment(
#     experiment_folder_path="../data/06_models/mlruns",
#     experiment_name="velib_prediction"
# )
# experiment_id

experiment_id = '587291553688351204'

In [14]:
search_params = {
    "iterations": (100, 500),
    "depth": (2, 7),
}

In [15]:
train_model_bayesian_opti(
    run_name="Test_catboost",
    experiment_id=experiment_id,
    search_params=search_params,
    df_train=df_train,
    df_valid=df_valid,
    feat_cat=feat_cat,
    n_trials=10
)

[I 2024-11-22 08:38:36,379] A new study created in memory with name: 


  0%|          | 0/10 [00:00<?, ?it/s]

[W 2024-11-22 08:38:36,392] Trial 0 failed with parameters: {} because of the following error: TypeError("optimize_hyperparams() got multiple values for argument 'experiment_id'").
Traceback (most recent call last):
  File "/Users/benjaminwallyn/Git/velib-prediction/.venv/lib/python3.11/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
TypeError: optimize_hyperparams() got multiple values for argument 'experiment_id'
[W 2024-11-22 08:38:36,393] Trial 0 failed with value None.


TypeError: optimize_hyperparams() got multiple values for argument 'experiment_id'